In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [4]:
shortforms = ['FES']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [5]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('functional electrical stimulation', 217),
 ('fat embolism syndrome', 28),
 ('first episode schizophrenia', 14),
 ('fluoroestradiol', 12),
 ('floppy eyelid syndrome', 10),
 ('free energy surface', 9),
 ('field electrical stimulation', 3),
 ('first episode of schizophrenia', 3),
 ('fluoro 17β estradiol', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [7]:
grounding_map, names, pos_labels = ({'fluoro 17β estradiol': 'PUBCHEM:10869981',
  'fat embolism syndrome': 'MESH:D004620',
  'first episode schizophrenia': 'ungrounded',
  'first episode of schizophrenia': 'ungrounded',
  'floppy eyelid syndrome': 'ungrounded',
  'fluoroestradiol': 'PUBCHEM:10869981',
  'free energy surface': 'ungrounded',
  'functional electrical stimulation': 'MESH:D004599',
  'field electrical stimulation': 'MESH:D004599'},
 {'PUBCHEM:10869981': 'Fluoroestradiol F-18',
  'MESH:D004620': 'Embolism, Fat',
  'MESH:D004599': 'Electric Stimulation Therapy',
  'HGNC:3657': 'FES'},
 ['MESH:D004599', 'MESH:D004620', 'PUBCHEM:10869981', 'HGNC:3657'])

In [8]:
excluded_longforms = []

In [9]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [10]:
additional_entities = {}

In [11]:
unambiguous_agent_texts = {}

In [12]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [13]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [14]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [15]:
intersection1

[]

In [16]:
intersection2

[]

In [17]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [18]:
names.update({key: value[0] for key, value in additional_entities.values()})
pos_labels = list(set(pos_labels) | additional_entities.keys())

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 01:50:36] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 01:50:39] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9601613893681951 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'PUBCHEM:10869981': 11,
  'MESH:D004599': 145,
  'MESH:D004620': 17,
  'ungrounded': 23},
 'f1': {'mean': 0.960161, 'std': 0.030209},
 'precision': {'mean': 0.946955, 'std': 0.033215},
 'recall': {'mean': 0.982689, 'std': 0.022943},
 'PUBCHEM:10869981': {'f1': {'mean': 0.9, 'std': 0.2},
  'pr': {'mean': 0.866667, 'std': 0.266667},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.813492, 'std': 0.144915},
  'pr': {'mean': 0.71, 'std': 0.200998},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D004599': {'f1': {'mean': 0.967102, 'std': 0.020649},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.937083, 'std': 0.039207}},
 'MESH:D004620': {'f1': {'mean': 0.96, 'std': 0.08},
  'pr': {'mean': 0.933333, 'std': 0.133333},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [22]:
disamb.dump(model_name, results_path)

In [23]:
print(disamb.info())

Disambiguation model for FES

Produces the disambiguations:
	Electric Stimulation Therapy*	MESH:D004599
	Embolism, Fat*	MESH:D004620
	FES*	HGNC:3657
	Fluoroestradiol F-18*	PUBCHEM:10869981

Class level metrics:
--------------------
Grounding                   	Count	F1     
Electric Stimulation Therapy*	145	 0.9671
                  Ungrounded	 23	0.81349
               Embolism, Fat*	 17	   0.96
        Fluoroestradiol F-18*	 11	    0.9

Weighted Metrics:
-----------------
	F1 score:	0.96016
	Precision:	0.94696
	Recall:		0.98269

* Positive labels
See Docstring for explanation



In [24]:
model_to_s3(disamb)